# ID2214/FID3214 Assignment 3 Group no. []
### Project members: 
[Enter Name, email]
[]
[Enter Name, email]
[Enter Name, email]

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas, time and sklearn.tree, may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas, time and DecisionTreeClassifier from sklearn.tree

In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier

## Reused functions from Assignment 1

In [5]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

# filter away the columns with only na value or only one unique value
def create_column_filter(df):
    # use dropna, how='all' remove the columns with all NaN values
    filtered_df = df.dropna(axis=1, how='all')
    
    # use apply to cal the unique values in columns and find columns where this value==1
    nunique = df.apply(pd.Series.nunique)
    same_value_cols = nunique[nunique == 1].index
    filtered_df = filtered_df.drop(same_value_cols, axis=1)
    
    column_filter = filtered_df.columns.tolist()
    return filtered_df, column_filter

def apply_column_filter(df, column_filter):
    return df[column_filter]

def create_imputation(df):
    imputation = dict()
    for i in df.columns:
        if i != "CLASS" and i != "ID":
            if len(df[i].dropna()) == 0:
                if df[i].dtype == "float64" or df[i].dtype == "int64":
                    imputation[i] = 0
                elif df[i].dtype == "object":
                    imputation[i] = ""
                else: # df[i].dtype == "category":
                    imputation[i] = df[i].cat.categories[0]
            elif df[i].dtype == "float64" or df[i].dtype == "int64":
                imputation[i] = df[i].mean()
            else: # df[i].dtype == "object" or df[i].dtype == "category"
                imputation[i] = df[i].mode().iloc[0]  # multiple modes can exist, mode() should return a dataframe
    df = df.fillna(value=imputation)
    return df, imputation

def apply_imputation(df, imputation):
    df = df.fillna(value=imputation)
    return df

def create_normalization(df, normalizationtype="minmax"):
    normalization = {}
    if normalizationtype=="minmax":        
        for i in df.columns:
            if i != "CLASS" and i != "ID" and (df[i].dtype == "int64" or df[i].dtype == "float64"):       
                temp_min = df[i].min()
                temp_max = df[i].max()
                df[i] = [(x-temp_min)/(temp_max-temp_min) for x in df[i]]
                normalization[i] = (normalizationtype, temp_min, temp_max)
    else: # normalizationtype=="zscore"
        for i in df.columns:
            if i != "CLASS" and i != "ID" and (df[i].dtype == "int64" or df[i].dtype == "float64"):
                temp_mean = df[i].mean()
                temp_std = df[i].std()
                df[i] = [(x-temp_mean)/std for x in df[i]]
                normalization[i] = (normalizationtype, temp_mean, temp_std)
    return df, normalization

def apply_normalization(df, normalization):
    for i in normalization:
        df[i] = [(x-normalization[i][1]) / (normalization[i][2]-normalization[i][1]) for x in df[i]]
    return df

from pandas.api.types import is_numeric_dtype
def create_one_hot(df):
    one_hot = {}
    for col in df.columns:
        if col != 'ID' and col != 'CLASS':
            if not is_numeric_dtype(df[col]):
                df[col] = df[col].astype("category")
                one_hot[col] = list(df[col].cat.categories)
                for i in one_hot[col]:
                    new_col = df[col]==i
                    new_col = new_col.astype("float")
                    df[col + "_" + i] = new_col 
                df = df.drop(columns = col, axis = 1)
    return df, one_hot

def apply_one_hot(df,one_hot):
    for col in df.columns:
        if col in one_hot.keys():    
            for i in one_hot[col]:
                new_col = df[col]==i
                new_col = pd.Series(new_col.astype("float"))
                df[col + "_" + i] = new_col
            df = df.drop(columns = col, axis = 1)
            
    return df   

import math
def folds(df, nofolds=10):
    folds = []
    index = np.random.permutation(df.index)
    num = math.ceil(len(index)/nofolds)
    for i in range(nofolds):
        folds.append(df.loc[index[num*i: num*(i+1)]])
        
    return folds

def accuracy(df, correctlabels):
    true_cases = 0
    all_cases = 0
    for i in df.index:
        all_cases += 1
        max_index = df.iloc[i].idxmax()
        if max_index == correctlabels[i]:
            true_cases += 1
    return true_cases / all_cases

# def brier_score(df, correctlabels):
#     bs_sum = 0
#     bs_N = 0
#     for i in df.index: # 0, 1, 2, 3, 4
#         for j in df.columns: # A, B, C
#             if correctlabels[i]==j:
#                 bs_sum += (df.ix[i][j]-1)**2
#             else:
#                 bs_sum += (df.ix[i][j]-0)**2
#         bs_N += 1
    
#     brier_score = bs_sum / bs_N
#     return brier_score

def brier_score(df, correctlabels):
    df = df.copy()
    brier_score = 0
    for i in df.index:
        p = df.iloc[i]
        o_index = np.where(df.columns==correctlabels[i])[0]
        p.iloc[o_index] -= 1
        brier_score += np.dot(p, p)
    brier_score = brier_score / len(correctlabels)
    return brier_score

def auc(df,correctlabels):
    correctlabels = correctlabels.tolist()
    new_df=df.copy()
    lens=len(correctlabels)
    cols=new_df.columns.tolist()
    if len(cols)<3: #binary ;only Pos and Neg
        pos=[ i for i in range(lens)  if correctlabels[i]==cols[0]]
        neg=[i for i in range(lens)  if correctlabels[i]!=cols[0]]                    
#         print(pos,neg)
        one=new_df[cols[0]]
        auc = 0
        for i in pos:
            for j in neg:
                if one[i] > one[j]:
                    auc += 1
                elif one[i] == one[j]:
                    auc += 0.5

        return auc / (len(pos)*len(neg))
    else:      # CLASS >=3   Pos / non-Pos
        aucs=[]
        for col in cols:
            pos=[ i for i in range(lens)  if correctlabels[i]==col]
            nonpos=[i for i in range(lens)  if correctlabels[i]!=col]                      
#             print(pos,nonpos)
            one=new_df[col]
            auc = 0
            for i in pos:
                for j in nonpos:
                    if one[i] > one[j]:
                        auc += 1
                    elif one[i] == one[j]:
                        auc += 0.5
            auc=auc/(len(pos)*len(nonpos))
            aucs.append(auc)
        weights=[correctlabels.count(col)/len(correctlabels) for col in cols ]
#         print(weights)
#         print(aucs)

        weights=np.array(weights)
        aucs=np.array(aucs)
        avg_auc=np.sum(aucs*weights)
        return avg_auc


## 1. Define the class RandomForest

In [67]:
# Define the class RandomForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, one_hot, labels, model
#
class RandomForest:
    
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.training_labels=None
        self.model = None

# Input to fit:
# self      - the object itself
# df        - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# no_trees  - no. of trees in the random forest (default = 100)
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter - a column filter (see Assignment 1) from df
# self.imputation    - an imputation mapping (see Assignment 1) from df
# self.one_hot       - a one-hot mapping (see Assignment 1) from df
# self.labels        - a (sorted) list of the categories of the "CLASS" column of df
# self.model         - a random forest, consisting of no_trees trees, where each tree is generated from a bootstrap sample
#                      and the number of evaluated features is log2|F| where |F| is the total number of features
#                      (for details, see lecture slides)
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First create the column filter, imputation and one-hot mappings
#
# Hint 2: Then get the class labels and the numerical values (as an ndarray) from the dataframe after dropping the class labels 
#
# Hint 3: Generate no_trees classification trees, where each tree is generated using DecisionTreeClassifier 
#         from a bootstrap sample (see lecture slides), e.g., generated by np.random.choice (with replacement) 
#         from the row numbers of the ndarray, and where a random sample of the features are evaluated in
#         each node of each tree, of size log2(|F|), where |F| is the total number of features;
#         see the parameter max_features of DecisionTreeClassifier
    
    def fit(self, dataframe, no_trees=100):
        df = dataframe.copy()
        df, self.column_filter  = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.one_hot = create_one_hot(df)
        df["CLASS"] = df["CLASS"].astype("category")
        self.training_labels = df["CLASS"]
        self.labels = list(df["CLASS"].cat.categories)
        self.training_data = df.drop(columns=["ID", "CLASS"], errors='ignore')
        
        models=[]
        
        sum=len(self.training_data)
        size=sum  #  how big the size of per sample should be
        
        for i in range(no_trees):
            row_nums=np.random.choice(sum,size)
            #print(row_nums)
            X=self.training_data.loc[row_nums]
            label=[self.training_labels[i] for i in row_nums]
            #print(input)
            DT=DecisionTreeClassifier(max_features='log2')
            DT.fit(X,label)
            models.append(DT)
        
        self.model=models
        #print(self.training_labels,self.training_data)


# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are the averaged probabilities output by each decision tree in the forest
#
# Hint 1: Drop any "CLASS" and "ID" columns of the dataframe first and then apply column filter, imputation and one_hot
#
# Hint 2: Iterate over the trees in the forest to get the prediction of each tree by the method predict_proba(X) where 
#         X are the (numerical) values of the transformed dataframe; you may get the average predictions of all trees,
#         by first creating a zero-matrix with one row for each test instance and one column for each class label, 
#         to which you add the prediction of each tree on each iteration, and then finally divide the prediction matrix
#         by the number of trees.
#
# Hint 3: You may assume that each bootstrap sample that was used to generate each tree has included all possible
#         class labels and hence the prediction of each tree will contain probabilities for all class labels
#         (in the same order). Note that this assumption may be violated, and this limitation will be addressed 
#         in the next part of the assignment. 

    def predict(self, dataframe):
        df = dataframe.copy()
        df = apply_column_filter(df, self.column_filter)
        df = apply_imputation(df, self.imputation)
        df = apply_one_hot(df,self.one_hot)
        labels = self.labels # class categories
        df.drop(columns=["CLASS"], inplace=True)
        
        row,col=len(df),len(labels)
        
        probs=np.zeros((row,col))
        
        for cur_DT in self.model:
            for i in range(row):
                cur_test=df.loc[i].values.reshape(1,-1)
                result=cur_DT.predict_proba(cur_test)
                probs[i]=probs[i]+result
                #print(probs[i])
        #print(probs)
        
        probs=probs/len(self.model)
        predictions = pd.DataFrame(probs, columns=labels)
        #print(predictions)
        return predictions

In [68]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("tic-tac-toe_train.csv")

test_df = pd.read_csv("tic-tac-toe_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)

print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 2.74 s.
     negative  positive
0        0.30      0.70
1        0.19      0.81
2        0.10      0.90
3        0.50      0.50
4        0.18      0.82
5        0.14      0.86
6        0.03      0.97
7        0.48      0.52
8        0.07      0.93
9        0.11      0.89
10       0.02      0.98
11       0.67      0.33
12       0.33      0.67
13       0.65      0.35
14       0.20      0.80
15       0.40      0.60
16       0.62      0.38
17       0.71      0.29
18       0.04      0.96
19       0.12      0.88
20       0.78      0.22
21       0.36      0.64
22       0.13      0.87
23       0.21      0.79
24       0.44      0.56
25       0.12      0.88
26       0.39      0.61
27       0.21      0.79
28       0.66      0.34
29       0.60      0.40
..        ...       ...
449      0.31      0.69
450      0.27      0.73
451      0.42      0.58
452      0.70      0.30
453      0.18      0.82
454      0.61      0.39
455      0.13      0.87
456      0.21      0.79
457      0.46    

In [69]:
train_labels = train_df["CLASS"]
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels))) # Comment this out if not implemented in assignment 1
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels))) # Comment this out if not implemented in assignment 1

     negative  positive
0        0.90      0.10
1        0.78      0.22
2        0.96      0.04
3        0.03      0.97
4        0.07      0.93
5        0.82      0.18
6        0.04      0.96
7        0.07      0.93
8        0.07      0.93
9        0.96      0.04
10       0.02      0.98
11       0.02      0.98
12       0.03      0.97
13       0.02      0.98
14       0.06      0.94
15       0.05      0.95
16       0.10      0.90
17       0.81      0.19
18       0.90      0.10
19       0.92      0.08
20       0.08      0.92
21       0.93      0.07
22       0.83      0.17
23       0.81      0.19
24       0.77      0.23
25       0.08      0.92
26       0.83      0.17
27       0.06      0.94
28       0.91      0.09
29       0.87      0.13
..        ...       ...
449      0.84      0.16
450      0.87      0.13
451      0.08      0.92
452      0.04      0.96
453      0.72      0.28
454      0.01      0.99
455      0.74      0.26
456      0.88      0.12
457      0.94      0.06
458      0.04   

### Comment on assumptions, things that do not work properly, etc.


## 2a. Handling trees with non-aligned predictions

In [99]:
# Define a revised version of the class RandomForest with the same input and output as described in part 1 above,
# where the predict function is able to handle the case where the individual trees are trained on bootstrap samples
# that do not include all class labels in the original training set. This leads to that the class probabilities output
# by the individual trees in the forest do not refer to the same set of class labels.
#
# Hint 1: The categories obtained with <pandas series>.cat.categories are sorted in the same way as the class labels
#         of a DecisionTreeClassifier; the latter are obtained by <DecisionTreeClassifier>.classes_ 
#         The problem is that classes_ may not include all possible labels, and hence the individual predictions 
#         obtained by <DecisionTreeClassifier>.predict_proba may be of different length or even if they are of the same
#         length do not necessarily refer to the same class labels. You may assume that each class label that is not included
#         in a bootstrap sample should be assigned zero probability by the tree generated from the bootstrap sample. 
#
# Hint 2: Create a mapping from the complete (and sorted) set of class labels l0, ..., lk-1 to a set of indexes 0, ..., k-1,
#         where k is the number of classes
#
# Hint 3: For each tree t in the forest, create a (zero) matrix with one row per test instance and one column per class label,
#         to which one column is added at a time from the output of t.predict_proba 
#
# Hint 4: For each column output by t.predict_proba, its index i may be used to obtain its label by t.classes_[i];
#         you may then obtain the index of this label in the ordered list of all possible labels from the above mapping (hint 2); 
#         this index points to which column in the prediction matrix the output column should be added to 

class RandomForest:
    
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.training_labels=None
        self.model = None

    def fit(self, dataframe, no_trees=100):
        df = dataframe.copy()
        df, self.column_filter  = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.one_hot = create_one_hot(df)
        df["CLASS"] = df["CLASS"].astype("category")
        self.training_labels = df["CLASS"]
        self.labels = list(df["CLASS"].cat.categories)
        #print(self.labels)
        self.label_mapping=dict(zip(self.labels,list(range(len(self.labels)))))
        self.training_data = df.drop(columns=["ID", "CLASS"], errors='ignore')
        
#         print(self.label_mapping)
#         print(self.training_data)
        models=[]
        
        sum=len(self.training_data)
        size=sum  #  how big the size of per sample should be
        
        for i in range(no_trees):
            row_nums=np.random.choice(sum,size)
            #print(row_nums)
            X=self.training_data.loc[row_nums]
            label=[self.training_labels[i] for i in row_nums]
            #print(input)
            DT=DecisionTreeClassifier(max_features='log2')
            DT.fit(X,label)
            models.append(DT)
        
        self.model=models
        #print(self.training_labels,self.training_data)
        
    def predict(self, dataframe):
        df = dataframe.copy()
        df = apply_column_filter(df, self.column_filter)
        df = apply_imputation(df, self.imputation)
        df = apply_one_hot(df,self.one_hot)
        labels = self.labels # class categories
        df.drop(columns=["CLASS"], inplace=True)
        
        row,col=len(df),len(labels)
        
        probs=np.zeros((row,col))
        
        for cur_DT in self.model:
            for i in range(row):
                cur_test=df.loc[i].values.reshape(1,-1)
                Cur_classes=cur_DT.classes_
                result=cur_DT.predict_proba(cur_test)[0]
#                 print(Cur_classes)
#                 print(result)
                for index,c in enumerate(Cur_classes):
                    j=self.label_mapping[c]
                    probs[i][j]=probs[i][j]+result[index]
        
        probs=probs/len(self.model)
        predictions = pd.DataFrame(probs, columns=labels)
        #print(predictions)
        return predictions

In [100]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 2.73 s.
        1      2      3     5     U
0    0.00  0.000  1.000  0.00  0.00
1    0.00  0.060  0.940  0.00  0.00
2    0.00  0.030  0.970  0.00  0.00
3    0.00  0.210  0.790  0.00  0.00
4    0.00  0.000  1.000  0.00  0.00
5    0.00  0.000  0.980  0.02  0.00
6    0.00  0.010  0.170  0.82  0.00
7    0.00  0.000  0.970  0.03  0.00
8    0.00  0.000  0.970  0.03  0.00
9    0.00  0.000  0.230  0.77  0.00
10   0.00  0.000  0.300  0.70  0.00
11   0.00  0.000  1.000  0.00  0.00
12   0.00  0.140  0.710  0.15  0.00
13   0.00  0.000  1.000  0.00  0.00
14   0.00  0.010  0.990  0.00  0.00
15   0.00  0.790  0.210  0.00  0.00
16   0.00  0.000  1.000  0.00  0.00
17   0.00  0.000  0.100  0.00  0.90
18   0.00  0.080  0.920  0.00  0.00
19   0.00  0.000  0.070  0.93  0.00
20   0.00  0.000  1.000  0.00  0.00
21   0.00  0.060  0.890  0.02  0.03
22   0.00  0.000  0.990  0.00  0.01
23   0.00  0.000  0.960  0.02  0.02
24   0.00  0.000  0.990  0.00  0.01
25   0.01  0.000  0.990  0.00  0.00
26   

## 2b. Estimate predictive performance using out-of-bag predictions

In [155]:
# Define an extended version of the class RandomForest with the same input and output as described in part 2a above,
# where the results of the fit function also should include:
# self.oob_acc - the accuracy estimated on the out-of-bag predictions, i.e., the fraction of training instances for 
#                which the given (correct) label is the same as the predicted label when using only trees for which
#                the instance is out-of-bag
#
# Hint 1: You may first create a zero matrix with one row for each training instance and one column for each class label
#         and one zero vector to allow for storing aggregated out-of-bag predictions and the number of out-of-bag predictions
#         for each training instance, respectively
#
# Hint 2: After generating a tree in the forest, iterate over the indexes that were not included in the bootstrap sample
#         and add a prediction of the tree to the out-of-bag prediction matrix and update the count vector
#
# Hint 3: Note that the input to predict_proba has to be a matrix; from a single vector (row) x, a matrix with one row
#         can be obtained by x[None,:]
#
# Hint 4: Finally, divide each row in the out-of-bag prediction matrix with the corresponding element of the count vector

class RandomForest:
    
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.one_hot = None
        self.labels = None
        self.training_labels=None
        self.model = None

    def fit(self, dataframe, no_trees=100):
        df = dataframe.copy()
        df, self.column_filter  = create_column_filter(df)
        df, self.imputation = create_imputation(df)
        df, self.one_hot = create_one_hot(df)
        df["CLASS"] = df["CLASS"].astype("category")
        self.training_labels = df["CLASS"]
        self.labels = list(df["CLASS"].cat.categories)
        #print(self.labels)
        self.label_mapping=dict(zip(self.labels,list(range(len(self.labels)))))
        self.training_data = df.drop(columns=["ID", "CLASS"], errors='ignore')
        
#         print(self.label_mapping)
#         print(self.training_data)
        models=[]
        
        sum=len(self.training_data)
        size=sum  #  how big the size of per sample should be
        
        missing_num_trees=[]
        
        for i in range(no_trees):
            row_nums=np.random.choice(sum,size)
            #print(row_nums)
            X=self.training_data.loc[row_nums]
            label=[self.training_labels[i] for i in row_nums]
            #print(input)
            DT=DecisionTreeClassifier(max_features='log2')
            DT.fit(X,label)
            models.append(DT)
            
            #oob
            #print(row_nums)
            missing_num=[i for i in range(sum) if i not in np.unique(row_nums)]
            #print(missing_num)
            missing_num_trees.append(missing_num)
        
        self.missing_num_trees=missing_num_trees
        self.model=models
        #print(self.training_labels,self.training_data)
        self.oob_acc=self.cal_acc(self.missing_num_trees)
    
    def cal_acc(self,mn_trees):       #low acc  problem?
            row,col=len(self.training_data),len(self.labels)
            oob_probs=np.zeros((row,col))
            counts=np.zeros(row)
            oob_models=self.model
            no_trees=len(mn_trees)
            
            for i in range(no_trees):
                nums=mn_trees[i]
                #print(nums)
                cur_model=oob_models[i]
                Cur_classes=cur_model.classes_
                
                for row in nums:
                    cur_train=self.training_data.loc[row].values.reshape(1,-1)
                    result=cur_model.predict_proba(cur_train)[0]
                    for index,c in enumerate(Cur_classes):
                        j=self.label_mapping[c]
                        oob_probs[row][j]=oob_probs[row][j]+result[index]
                    counts[row]+=1
#             print(oob_probs)
#             print(counts)      
            
            acc=oob_probs/counts[:,None]
            acc=pd.DataFrame(acc, columns=self.labels)
            result=accuracy(acc,self.training_labels)
            return result
        
    def predict(self, dataframe):
        df = dataframe.copy()
        df = apply_column_filter(df, self.column_filter)
        df = apply_imputation(df, self.imputation)
        df = apply_one_hot(df,self.one_hot)
        labels = self.labels # class categories
        df.drop(columns=["CLASS"], inplace=True)
        
        row,col=len(df),len(labels)
        
        probs=np.zeros((row,col))
        
        
        for cur_DT in self.model:
            for i in range(row):
                cur_test=df.loc[i].values.reshape(1,-1)
                Cur_classes=cur_DT.classes_
                result=cur_DT.predict_proba(cur_test)[0]
#                 print(Cur_classes)
#                 print(result)
                for index,c in enumerate(Cur_classes):
                    j=self.label_mapping[c]
                    probs[i][j]=probs[i][j]+result[index]
        
        probs=probs/len(self.model)
        predictions = pd.DataFrame(probs, columns=labels)
        #print(predictions)
        return predictions

In [156]:
# Test your code (leave this part unchanged, except for if auc is undefined)

train_df = pd.read_csv("anneal_train.csv")

test_df = pd.read_csv("anneal_test.csv")

rf = RandomForest()

t0 = time.perf_counter()
rf.fit(train_df)
print("Training time: {:.2f} s.".format(time.perf_counter()-t0))

print("OOB accuracy: {:.4f}".format(rf.oob_acc))

test_labels = test_df["CLASS"]

t0 = time.perf_counter()
predictions = rf.predict(test_df)
print("Testing time: {:.2f} s.".format(time.perf_counter()-t0))

print("Accuracy: {:.4f}".format(accuracy(predictions,test_labels)))
print("AUC: {:.4f}".format(auc(predictions,test_labels))) # Comment this out if not implemented in assignment 1
print("Brier score: {:.4f}".format(brier_score(predictions,test_labels))) # Comment this out if not implemented in assignment 1

Training time: 8.08 s.
OOB accuracy: 0.9488
Testing time: 13.43 s.
Accuracy: 0.9510
AUC: 0.9694
Brier score: 0.0991


In [148]:
train_labels = train_df["CLASS"]
rf = RandomForest()
rf.fit(train_df)
predictions = rf.predict(train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 1.00
AUC on training set: 1.00
Brier score on training set: 0.01


### Comment on assumptions, things that do not work properly, etc.